In [1]:
%load_ext autoreload
%autoreload 3
%reload_ext autoreload
import pandas as pd
import openpyxl
import re
from datetime import datetime
from pathlib import Path
import os
from io import BytesIO
from azure.storage.blob import BlobServiceClient
from utils import read_archived_pool_proj
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Function to get weeks and comments where value is 1
def get_weeks_and_comments(row):
    weeks = []
    comments = []
    row_idx = int(row.name.strip("N°"))
    col_idx = 1
    for col, value in row.items():
        if value == 1:
            weeks.append(col)
            cell = sheet[row_idx + 1][col_idx]
            comments.append(cell.comment.text if cell.comment else "")
        col_idx += 1
    return pd.Series({"weeks": weeks, "comments": comments})


def idx_to_pool_number(df):
    df = (
        df.reset_index()
        .rename(columns={"index": "pool_number"})
        .assign(pool_number=lambda x: x["pool_number"].str.strip("N°"))
    )
    return df


# Function to extract information from comments
def extract_info(comment):
    equipo_match = re.search(r"Equipo:\s*(\d+)", comment)
    ns_match = re.search(r"NS:\s*(#\w*-?\w*)", comment)

    equipo = equipo_match.group(1) if equipo_match else None
    ns = ns_match.group(1) if ns_match else None

    return equipo, ns


# Function to get the end date of a week
def get_week_end_date(week_str):
    year, week = week_str.split("-W")
    # Create a datetime object for the first day of the year
    first_day = datetime(int(year), 1, 1)
    # Calculate the start of the week (Monday)
    start_of_week = first_day + timedelta(
        days=(int(week) - 1) * 7 - first_day.weekday()
    )
    # Calculate the end of the week (Sunday)
    end_of_week = start_of_week + timedelta(days=6)
    return end_of_week.date()


def read_pool_component_arrivals():

    blob_service_client = BlobServiceClient(
        account_url=os.environ["AZURE_ACCOUNT_URL"],
        credential=os.environ["AZURE_SAS_TOKEN"],
    )
    blob_client = blob_service_client.get_blob_client(
        container=os.environ["AZURE_CONTAINER_NAME"],
        blob=f"{os.environ['AZURE_PREFIX']}/PLANIFICACION/POOL/Pool Componente MEL.xlsx",
    )
    blob_data = blob_client.download_blob()
    blob_data = BytesIO(blob_data.readall())
    # Read the Excel file
    wb = openpyxl.load_workbook(blob_data, data_only=False)
    sheet = wb.active

    # Create a DataFrame from the Excel data
    data = []
    for row in sheet.iter_rows(
        min_row=36,
        max_row=43,
        values_only=True,
    ):
        data.append(row)

    df = pd.DataFrame(data, columns=[cell.value for cell in sheet[1]])

    df = df.dropna(how="all", axis=1)
    df.columns = df.iloc[0, :]
    df = df.iloc[1:, 1 : 2 + 52 * 4]

    # Generate date range
    date_range = pd.date_range(start="2022-01-01", end="2025-12-31", freq="W-MON")

    # Create the list of week strings
    week_list = [
        f"{date.isocalendar()[0]}-W{date.isocalendar()[1]:02d}" for date in date_range
    ]
    week_list = ["componente", *week_list[0 : 4 * 52]]
    df.columns = week_list
    df = df.dropna(how="all", axis=1)

    # Melt the dataframe
    df_melted = df.melt(
        id_vars=["componente"], var_name="arrival_week", value_name="n_components"
    ).dropna()
    # Convert n_components to integer
    df_melted["n_components"] = df_melted["n_components"].astype(int)

    # Apply the expand_rows function and reset the index
    df_melted["pool_slot"] = df_melted.apply(
        lambda row: list(range(row["n_components"])), axis=1
    )
    df = df_melted.explode("pool_slot")

    df = df.sort_values(["componente", "arrival_week"]).reset_index(drop=True)
    df = df.assign(
        component_code=lambda x: x["componente"]
        .str.strip(" ")
        .map(
            lambda x: {
                "Blower Parrilla": "bp",
                "Cilindro Dirección": "cd",
                "Suspensión Trasera": "st",
                "Suspensión Delantera": "cms",
                "Motor Tracción": "mt",
                "Cilindro Levante": "cl",
                "Modulo Potencia": "mp",
            }[x]
        ),
    ).drop(columns=["componente"])

    # Apply the function to create a new column
    df["arrival_date"] = pd.to_datetime(df["arrival_week"].apply(get_week_end_date))
    return df


# df = df.assign(
#     arrival_date=df["arrival_week"].map(
#         lambda x: datetime.strptime(x + "-6", "%Y-W%W-%w")
#     )
# )
arrivals_df = read_pool_component_arrivals()
arrivals_df
# df.set_index(df.columns[0], inplace=True)

# # Process the dataframe
# df = (
#     df.apply(get_weeks_and_comments, axis=1)
#     .explode(["weeks", "comments"])
#     .rename(columns={"weeks": "cc_week"})
#     .pipe(idx_to_pool_number)
# )

,arrival_week,n_components,pool_slot,component_code,arrival_date
0,2022-W45,1,0,bp,2022-11-06
1,2022-W47,2,0,bp,2022-11-20
2,2022-W47,2,1,bp,2022-11-20
3,2022-W49,1,0,bp,2022-12-04
4,2023-W02,1,0,bp,2023-01-08
...,...,...,...,...,...
351,2024-W22,1,0,st,2024-06-02
352,2024-W24,1,0,st,2024-06-16
353,2024-W27,1,0,st,2024-07-07
354,2024-W28,1,0,st,2024-07-14


In [3]:
blob_service_client = BlobServiceClient(
    account_url=os.environ["AZURE_ACCOUNT_URL"],
    credential=os.environ["AZURE_SAS_TOKEN"],
)

blob_client = blob_service_client.get_blob_client(
    container=os.environ["AZURE_CONTAINER_NAME"],
    blob=f"{os.environ['AZURE_PREFIX']}/PLANIFICACION/POOL/PLANILLA DE CONTROL CAMBIO DE COMPONENTES MEL.xlsx",
)
blob_data = blob_client.download_blob()
blob_data = BytesIO(blob_data.readall())
cc_df = pd.read_excel(blob_data)
columns_map = {
    "EQUIPO": "equipo",
    "COMPONENTE": "componente",
    "SUB COMPONENTE": "subcomponente",
    "POSICION": "position",
    "N/S RETIRADO": "component_serial",
    "W": "changeout_week",
    "FECHA DE CAMBIO": "changeout_date",
    "HORA CC": "component_hours",
    "TBO": "tbo_hours",
}
cc_df = (
    cc_df[list(columns_map.keys())]
    .dropna(subset=["COMPONENTE"])
    .rename(columns=columns_map)
    .assign(
        equipo=lambda x: x["equipo"].str.extract(r"(\d+)"),
        # changeout_week=lambda x: x["FECHA DE CAMBIO"]
        # .dt.year.astype(int)
        # .astype(str)
        # .str.cat(x["changeout_week"].str.zfill(2), sep="-W"),
        component_code=lambda x: x["componente"].map(
            lambda x: {
                "Blower": "bp",
                "Cilindro_Dirección": "cd",
                "Suspensión_Trasera": "st",
                "CMS": "cms",
                "Motor_Tracción": "mt",
                "Cilindro_Levante": "cl",
                "Módulo_Potencia": "mp",
            }.get(x)
        ),
    )
    .dropna(subset=["component_code"])
)
cc_df = cc_df.assign(
    changeout_week=lambda x: x["changeout_date"]
    .dt.year.astype(str)
    .str.cat(x["changeout_week"].astype(str), sep="-W")
)
cc_df = cc_df.assign(pool_changeout_type=None)
cc_df = cc_df.query("component_code == 'mt'")
cc_df = cc_df.loc[
    ~(
        (cc_df["component_code"] == "mt")
        & ~(cc_df["subcomponente"].str.contains("MOTOR TRACCIÓN"))
    )
]
cc_df

/home/cecilvega/DataspellProjects/koapp/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
0,862,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#W11110015,2019-W1,2019-01-01,14025.0,24000.0,mt,None
5,859,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX15020001T,2019-W1,2019-01-04,25017.0,24000.0,mt,None
31,847,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#W13020043,2019-W5,2019-01-30,15130.0,24000.0,mt,None
44,856,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX14090042T,2019-W7,2019-02-16,884.0,24000.0,mt,None
100,865,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#W12010005,2019-W15,2019-04-11,13761.0,24000.0,mt,None
...,...,...,...,...,...,...,...,...,...,...,...
2647,322,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#W13050087,2024-W23,2024-06-06,6620.0,24000.0,mt,None
2688,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None
2696,852,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX1410049T,2024-W29,2024-07-21,12339.0,24000.0,mt,None
2700,849,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX15030003T,2024-W30,2024-07-25,23208.0,24000.0,mt,None


In [7]:
pool_proj_df = (
    read_archived_pool_proj().drop(columns=["changeout_date"])
    # .rename(
    #     columns={
    #         "arrival_date": "arrival_date_archived",
    #         "arrival_week": "arrival_week_archived",
    #     }
    # )
)
pool_proj_df.query("pool_slot=='6' & component_code=='mt'")

,pool_slot,changeout_week,equipo,component_serial,arrival_week,pool_changeout_type,arrival_date,component_code
5,6,2023-W49,322,#WX14100052T,2024-W16,I,2024-04-15,mt
17,6,2024-W21,856,#WX14020007T,2024-W39,I,2024-09-23,mt


In [19]:
merge_columns = ["equipo", "component_code", "component_serial", "changeout_week"]
pool_slots_df = pd.merge(
    pool_proj_df.query("component_code == 'mt'"),
    cc_df,
    on=merge_columns,
    how="left",
    suffixes=("_proj", ""),
)
pool_slots_df = pool_slots_df.assign(
    pool_changeout_type=np.where(
        pool_slots_df["pool_changeout_type"].isnull(),
        pool_slots_df["pool_changeout_type_proj"],
        pool_slots_df["pool_changeout_type"],
    ),
)
assert pool_slots_df["changeout_date"].notnull().all()
# New changes that were not added into the pool
new_cc_df = (
    pd.merge(
        cc_df.loc[cc_df["changeout_date"] >= datetime(2024, 6, 1)],
        pool_proj_df[merge_columns],
        on=merge_columns,
        how="left",
        indicator=True,
    )
    .query("_merge == 'left_only'")
    .drop(columns="_merge")
)
new_cc_df

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
1,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None
2,852,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX1410049T,2024-W29,2024-07-21,12339.0,24000.0,mt,None
3,849,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX15030003T,2024-W30,2024-07-25,23208.0,24000.0,mt,None
4,285,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#W08031004,2024-W31,2024-07-29,28605.0,24000.0,mt,None


In [20]:
# Sort cc_df by changeout_date
new_cc_df = new_cc_df.sort_values("changeout_date")


def find_available_pool_slot(pool_slots_df, changeout_date):
    # Group by pool_slot and find the latest event for each
    latest_events = (
        pool_slots_df.groupby("pool_slot")
        .apply(
            lambda x: x.loc[
                (
                    x["changeout_date"].idxmax()
                    if not x["changeout_date"].isna().all()
                    else x["arrival_date"].idxmax()
                )
            ]
        )
        .reset_index(drop=True)
    )

    # Filter for available slots
    available_slots = latest_events[
        (
            (latest_events["changeout_date"] < changeout_date)
            & (latest_events["arrival_date"] < changeout_date)
        )
        | (latest_events["arrival_date"] < changeout_date)
    ]

    if available_slots.empty:
        return None

    return available_slots  # .loc[available_slots["days_unchanged"].idxmax()]


def find_most_time_unchanged_slot(df, changeout_date):
    # Find the slot with the most time unchanged
    df["days_unchanged"] = (changeout_date - df["arrival_date"]).dt.days
    return df.loc[df["days_unchanged"].idxmax()]


# def allocate_components(cc_df, pool_slots_df):
for _, changeout in new_cc_df.iloc[0:1].iterrows():
    available_slots_df = find_available_pool_slot(
        pool_slots_df, changeout["changeout_date"]
    )
    available_slot = find_most_time_unchanged_slot(
        available_slots_df, changeout["changeout_date"]
    )
    if available_slot is None:
        print(
            f"Unable to find an available pool slot for changeout on {changeout['changeout_date']}"
        )
        # return pool_slots_df
    # Add new row to pool_slots_df
    new_row = changeout.copy()
    new_row["pool_slot"] = available_slot["pool_slot"]
    new_row["arrival_date"] = pd.NaT  # This will be set when the component returns

    pool_slots_df = pd.concat(
        [pool_slots_df, pd.DataFrame([new_row])], ignore_index=True
    )

    # Sort the dataframe to maintain chronological order
    pool_slots_df = pool_slots_df.sort_values("changeout_date")

pool_slots_df

/tmp/ipykernel_58098/2322142248.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,pool_slot,changeout_week,equipo,component_serial,arrival_week,pool_changeout_type_proj,arrival_date,component_code,componente,subcomponente,position,changeout_date,component_hours,tbo_hours,pool_changeout_type
0,1,2023-W29,866,#W12010011,2023-W48,I,2023-11-27,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-07-22,13892.0,24000.0,I
1,2,2023-W31,874,#WX15040005T,2023-W50,I,2023-12-11,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-08-04,10050.0,24000.0,I
2,3,2023-W39,875,#W12010016,2024-W6,I,2024-02-05,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-09-27,18937.0,24000.0,I
3,4,2023-W42,860,#W11110015,2024-W9,I,2024-02-26,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-10-20,3406.0,24000.0,I
4,5,2023-W43,860,#WX1606Y004T,2024-W10,I,2024-03-04,mt,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,2023-10-23,3379.0,24000.0,I
5,6,2023-W49,322,#WX14100052T,2024-W16,I,2024-04-15,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-12-08,4630.0,24000.0,I
6,7,2023-W50,857,#W12030036,2024-W17,I,2024-04-22,mt,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,2023-12-14,10234.0,24000.0,I
7,8,2023-W51,855,#WX14100053T,2024-W18,I,2024-04-29,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2023-12-20,11162.0,24000.0,I
8,9,2024-W6,869,#W12030041,2024-W25,I,2024-06-17,mt,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,2024-02-10,6770.0,24000.0,I
9,10,2024-W9,877,#WX14080032T,2024-W28,I,2024-07-08,mt,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,2024-02-28,17529.0,24000.0,I


In [15]:
available_slot

pool_slot                                     8
changeout_week                         2023-W51
equipo                                      855
component_serial                   #WX14100053T
arrival_week                           2024-W18
pool_changeout_type_proj                      I
arrival_date                2024-04-29 00:00:00
component_code                               mt
componente                       Motor_Tracción
subcomponente                   MOTOR TRACCIÓN 
position                              IZQUIERDO
changeout_date              2023-12-20 00:00:00
component_hours                         11162.0
tbo_hours                               24000.0
pool_changeout_type                           I
days_unchanged                               77
Name: 10, dtype: object

In [108]:
new_cc_df

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
1,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None
2,852,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#WX1410049T,2024-W29,2024-07-21,12339.0,24000.0,mt,None
3,849,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX15030003T,2024-W30,2024-07-25,23208.0,24000.0,mt,None
4,285,Motor_Tracción,MOTOR TRACCIÓN,DERECHO,#W08031004,2024-W31,2024-07-29,28605.0,24000.0,mt,None


In [107]:
new_cc_df.iloc[0:1]

,equipo,componente,subcomponente,position,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type
1,872,Motor_Tracción,MOTOR TRACCIÓN,IZQUIERDO,#WX1902Y002T,2024-W29,2024-07-15,27730.0,24000.0,mt,None


In [71]:
df = pd.merge(
    pool_proj_df,
    cc_df,
    on=["equipo", "component_code", "component_serial", "changeout_week"],
    how="left",
    suffixes=("", "_proj"),
)
# Recuperar tipo de evento de cambio de componente
df = df.assign(pool_changeout_type=None)
df = df.assign(
    pool_changeout_type=np.where(
        df["pool_changeout_type"].isnull(),
        df["pool_changeout_type_proj"],
        df["pool_changeout_type"],
    ),
)
df = df.assign(
    ohv_normal=df["component_code"].map(
        lambda x: {
            "bp": 51,
            "cd": 46,
            "st": 65,
            "cms": 64,
            "mt": 74,
            "cl": 75,
            "mp": 64,
        }[x]
    ),
    ohv_unplanned=df["component_code"].map(
        lambda x: {
            "bp": 101,
            "cd": 96,
            "st": 125,
            "cms": 124,
            "mt": 134,
            "cl": 135,
            "mp": 114,
        }[x]
    ),
)
df = df.assign(
    proj_arrival_date=df["changeout_date"]
    + np.where(
        df["pool_changeout_type"] == "P",
        pd.to_timedelta(df["ohv_normal"], "D"),
        pd.to_timedelta(df["ohv_unplanned"], "D"),
    )
)

df = df.assign(pool_changeout_type=df["pool_changeout_type"].fillna("P"))
df = df.loc[df["component_code"] == "bp"]
df

,equipo,componente,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type,pool_slot_archived,arrival_week_archived,pool_changeout_type_proj,arrival_date_archived,ohv_normal,ohv_unplanned,proj_arrival_date
4,860,Blower,#EE13111400,2019-W1,2019-01-01,26007.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-12
9,866,Blower,#EE07040188,2019-W2,2019-01-10,12558.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-21
20,869,Blower,#EE14010117,2019-W3,2019-01-17,2577.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28
21,852,Blower,#EN12100128,2019-W3,2019-01-17,10476.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28
39,862,Blower,#EN12070112,2019-W6,2019-02-05,14267.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,879,Blower,#EE09100668,2024-W27,2024-07-01,3148.0,20000.0,bp,P,5,2024-W34,P,2024-08-19,51,101,2024-08-21
2637,322,Blower,#EN12040115,2024-W28,2024-07-09,4490.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-18
2643,320,Blower,#EE13101272,2024-W29,2024-07-15,13291.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-24
2660,398,Blower,#EE16030252,2024-W30,2024-07-25,7398.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-03


In [72]:
df = pd.merge_asof(
    df.sort_values("proj_arrival_date"),
    arrivals_df.sort_values("arrival_date"),
    by="component_code",
    left_on="proj_arrival_date",
    right_on="arrival_date",
    direction="nearest",
).sort_values("changeout_date", ascending=False)
df

,equipo,componente,component_serial,changeout_week,changeout_date,component_hours,tbo_hours,component_code,pool_changeout_type,pool_slot_archived,arrival_week_archived,pool_changeout_type_proj,arrival_date_archived,ohv_normal,ohv_unplanned,proj_arrival_date,arrival_week,n_components,pool_slot,arrival_date
238,871,Blower,#EE13111400,2024-W30,2024-07-27,18412.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-05,2024-W31,1,0,2024-08-04
237,398,Blower,#EE16030252,2024-W30,2024-07-25,7398.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-11-03,2024-W31,1,0,2024-08-04
236,320,Blower,#EE13101272,2024-W29,2024-07-15,13291.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-24,2024-W31,1,0,2024-08-04
235,322,Blower,#EN12040115,2024-W28,2024-07-09,4490.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2024-10-18,2024-W31,1,0,2024-08-04
233,879,Blower,#EE09100668,2024-W27,2024-07-01,3148.0,20000.0,bp,P,5,2024-W34,P,2024-08-19,51,101,2024-08-21,2024-W31,1,0,2024-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,862,Blower,#EN12070112,2019-W6,2019-02-05,14267.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-05-17,2022-W45,1,0,2022-11-06
2,869,Blower,#EE14010117,2019-W3,2019-01-17,2577.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28,2022-W45,1,0,2022-11-06
3,852,Blower,#EN12100128,2019-W3,2019-01-17,10476.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-28,2022-W45,1,0,2022-11-06
1,866,Blower,#EE07040188,2019-W2,2019-01-10,12558.0,20000.0,bp,P,NaN,NaN,NaN,NaT,51,101,2019-04-21,2022-W45,1,0,2022-11-06


In [27]:
cc_df

,equipo,componente,SUB COMPONENTE,MODÉLO,POSICION,FECHA DE CAMBIO,cc_week,HORA EQ,HORA CC,MENOR AL 90% TBO,...,Equipo,Solicitud de montaje,OS,RMCARE,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado,component
0,862,Motor_Tracción,MOTOR TRACCIÓN,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200159313,56542,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
1,862,Motor_Tracción,Motor Eléctrico,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200214529,56543,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
2,862,Motor_Tracción,Freno Servicio Trasero,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200170908,56660,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
3,862,Motor_Tracción,Freno Estacionamiento,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,25516,14025.0,0.0,...,200174141,56544,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.443583,mt
4,860,Blower,Blower,960E-2,IZQUIERDO,2019-01-01,2019.0-W01,26904,26007.0,1.0,...,200217130,56541,NaN,Cerrado,NaN,NaN,NaN,34646.0,1.732300,bp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,285,Motor_Tracción,Freno Servicio Trasero,930E-4,DERECHO,2024-07-29,2024.0-W31,18837,28605.0,1.0,...,200342417,90355,18128007,Cerrado,TBO,NaN,NaN,34.0,0.001417,mt
2713,285,Motor_Tracción,Freno Estacionamiento,930E-4,DERECHO,2024-07-29,2024.0-W31,18837,28605.0,1.0,...,200214225,90356,18128008,Cerrado,TBO,NaN,NaN,34.0,0.001417,mt
2714,847,Cilindro_Dirección,Cilindro Dirección,960E-2,DERECHO,2024-07-29,2024.0-W31,73983,13681.0,1.0,...,200195258,90352,18128009,Cerrado,TBO,NaN,NaN,34.0,0.002833,cd
2715,397,Cilindro_Dirección,Cilindro Dirección,960E-2,IZQUIERDO,2024-07-31,2024.0-W31,69973,13673.0,1.0,...,200210295,90238,NaN,Abierto,TBO,NaN,NaN,0.0,0.000000,cd


In [46]:
pool_proj_df

,pool_slot,changeout_week,equipo,component_serial,changeout_date,arrival_week,pool_changeout_type,arrival_date,component
0,1,2023-W27,856,#EG08090003,2023-07-03,2023-W40,P,2023-10-02,bp
1,2,2023-W27,881,#EN12040115,2023-07-03,2023-W34,P,2023-08-21,bp
2,3,2023-W27,875,#EN11110073,2023-07-03,2023-W38,P,2023-09-18,bp
3,4,2023-W28,883,#EN12090007,2023-07-10,2023-W38,P,2023-09-18,bp
4,6,2023-W29,883,#EE06050200-1,2023-07-17,2023-W42,P,2023-10-16,bp
...,...,...,...,...,...,...,...,...,...
11,12,2024-W11,848,#156,2024-03-11,2024-W20,P,2024-05-13,st
12,1,2024-W16,869,#696R009,2024-04-15,2024-W34,P,2024-08-19,st
13,2,2024-W18,882,#RLTF718,2024-04-29,2024-W36,P,2024-09-02,st
14,3,2024-W19,882,#RLTA681,2024-05-06,2024-W37,P,2024-09-09,st


In [ ]:
df = pd.merge_asof(
    cc_df.sort_values("changeout_date"),
    pool_proj_df.sort_values("changeout_date"),
    by="changeout_date",
    on=["equipo", "component", "component_serial"],
)
# df = df.assign(pool_type=np.where(df["HORA CC"] > 0.9 * df["TBO"], "P", "I"))
df = df.assign(
    ohv_normal=df["componente"].map(
        lambda x: {
            "Blower": 51,
            "Cilindro Dirección": 46,
            "Suspensión Trasera": 65,
            "CMSD": 64,
            "Motor Tracción": 74,
            "Cilindro Levante": 75,
            "Módulo Potencia": 64,
        }[x]
    ),
    ohv_unplanned=df["componente"].map(
        lambda x: {
            "Blower": 101,
            "Cilindro Dirección": 96,
            "Suspensión Trasera": 125,
            "CMSD": 124,
            "Motor Tracción": 134,
            "Cilindro Levante": 135,
            "Módulo Potencia": 114,
        }[x]
    ),
    # cc_date=df["cc_week"].map(lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")),
)
# df = df.assign(
#     arrival_date=df["cc_date"]
#     + np.where(
#         df["pool_type"] == "P",
#         pd.to_timedelta(df["ohv_normal"], "D"),
#         pd.to_timedelta(df["ohv_unplanned"], "D"),
#     )
# )
df
# df.to_csv("planification-consolidated.csv", index=False)

In [52]:
cc_df

,EQUIPO,COMPONENTE,SUB COMPONENTE,MODÉLO,POSICION,FECHA DE CAMBIO,W,HORA EQ,HORA CC,MENOR AL 90% TBO,...,N/S INSTALADO,Equipo,Solicitud de montaje,OS,RMCARE,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado
0,TKD 862,Motor_Tracción,MOTOR TRACCIÓN,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#W13060060,200159313,56542,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
1,TKD 862,Motor_Tracción,Motor Eléctrico,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#EE14080521,200214529,56543,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
2,TKD 862,Motor_Tracción,Freno Servicio Trasero,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#154,200170908,56660,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
3,TKD 862,Motor_Tracción,Freno Estacionamiento,960E-2,IZQUIERDO,2019-01-01,1,25516,14025.0,0.0,...,#470,200174141,56544,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.442167
4,TKD 860,Blower,Blower,960E-2,IZQUIERDO,2019-01-01,1,26904,26007.0,1.0,...,#EE13101129,200217130,56541,NaN,Cerrado,NaN,NaN,NaN,34612.0,1.730600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4906,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4907,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4908,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4909,NaN,_,NaN,NaN,NaN,NaT,52,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df = pd.merge(archived_pool_df, cc_df, on=["pool_slot", "component"], how="left")
df

KeyError: 'pool_slot'

In [8]:
df

,pool_slot,changeout_week,equipo,ns,changeout_date,arrival_week,pool_changeout_type,arrival_date,component
0,1,2023-W28,874,#240,2023-07-10,2023-W45,I,2023-11-06,cms
1,2,2023-W30,875,#201-1,2023-07-24,2023-W46,I,2023-11-13,cms
2,4,2023-W31,859,#189,2023-07-31,2023-W49,I,2023-12-04,cms
3,3,2023-W31,854,#223,2023-07-31,2023-W49,I,2023-12-04,cms
4,5,2023-W34,858,#196,2023-08-21,2023-W42,P,2023-10-16,cms
5,6,2023-W35,878,#221,2023-08-28,2023-W43,P,2023-10-23,cms
6,7,2023-W36,398,#203,2023-09-04,2023-W45,P,2023-11-06,cms
7,8,2023-W37,859,#220,2023-09-11,2023-W46,P,2023-11-13,cms
8,9,2023-W39,856,#228,2023-09-25,2024-W2,I,2024-01-08,cms
9,10,2023-W40,875,#005,2023-10-02,2024-W3,I,2024-01-15,cms
